# Laporan Tugas Data Mining  
## Deteksi Anomali pada Dataset Credit Card Fraud  
### Pendekatan Statistical, Proximity, dan Density  

### **Identitas Kelompok 12**

| NRP        | Nama                           |
|------------|--------------------------------|
| 5054241029 | Wayan Raditya Putra            |
| 5054241026 | Arya Pratama Rhama Putra       |
| 5054231020 | Dean Arkeanu Andharu           |





## BAB 1 — Pendahuluan

Deteksi anomali (anomaly detection) adalah proses mengidentifikasi objek atau titik data 
yang menunjukkan perilaku menyimpang dari pola umum mayoritas. Teknik ini memiliki 
peran penting dalam berbagai sistem yang membutuhkan deteksi dini terhadap aktivitas
tidak wajar, seperti keamanan jaringan, monitoring kesehatan sistem, dan deteksi 
transaksi finansial yang berpotensi fraud.

Pada tugas ini, pendekatan anomaly detection diterapkan pada *Credit Card Fraud 
Detection Dataset* dari Kaggle. Dataset tersebut sering digunakan sebagai studi kasus 
karena karakteristiknya yang menantang dan relevan dengan skenario nyata, di mana 
transaksi fraud hanya merupakan sebagian kecil dari keseluruhan data, sehingga tidak 
ideal ditangani dengan supervised learning secara langsung.

Laporan ini bertujuan untuk:
1. Mempelajari konsep anomaly dan outlier sesuai Bab 10 dari buku *Data Mining: Concepts and Techniques* oleh Pang-Ning Tan.
2. Mengimplementasikan tiga pendekatan anomaly detection, yaitu:
   - Statistical Approach  
   - Proximity Approach  
   - Density Approach
3. Menerapkan ketiga pendekatan tersebut pada dataset Credit Card Fraud Detection.
4. Mengevaluasi performa setiap pendekatan menggunakan metrik akurasi, precision, recall, dan waktu komputasi.
5. Memberikan analisis perbedaan performa dan menarik kesimpulan mengenai metode yang paling sesuai untuk mendeteksi anomali pada skenario fraud detection.

Melalui laporan ini, diharapkan pembaca dapat memahami perbedaan karakteristik 
setiap pendekatan anomaly detection serta bagaimana penerapannya pada data dunia nyata.


## BAB 2 — Dasar Teori

### 2.1 Anomaly dan Outlier
Anomaly merupakan objek atau titik data yang memiliki karakteristik berbeda secara signifikan dibandingkan sebagian besar data lainnya. 
Dalam banyak aplikasi, anomali dapat merepresentasikan kejadian penting, seperti kesalahan sistem, aktivitas mencurigakan, atau potensi fraud.

Outlier adalah bentuk anomaly yang muncul sebagai titik penyimpang dalam distribusi data. 
Keberadaan outlier dapat disebabkan oleh noise, kesalahan pencatatan, atau indikasi kejadian nyata yang tidak mengikuti pola umum.

Deteksi anomaly bertujuan untuk mengidentifikasi titik-titik tersebut secara otomatis dengan berbagai pendekatan, baik yang berbasis statistik, jarak, maupun kepadatan data.


### 2.2 Statistical Approach
Pendekatan statistik didasarkan pada asumsi bahwa data normal mengikuti pola distribusi tertentu, umumnya distribusi Gaussian. 
Data yang berada jauh dari pusat distribusi atau memiliki probabilitas sangat kecil di bawah model probabilistik dianggap sebagai anomaly.

Metode populer dalam pendekatan ini adalah *Elliptic Envelope*, yang memodelkan data normal dalam bentuk kurva elips berdasarkan mean dan covariance. 
Titik data yang berada di luar batas elips tersebut diklasifikasikan sebagai outlier.

Pendekatan ini efektif jika distribusi data relatif teratur, namun sensitif terhadap data yang tidak memenuhi asumsi distribusi tertentu.


### 2.3 Proximity Approach
Pendekatan proximity mendeteksi anomaly berdasarkan jarak antar titik data. 
Ide dasarnya adalah bahwa titik yang jauh dari sebagian besar titik lain, atau memiliki sedikit tetangga dekat, cenderung merupakan outlier.

Contoh metode proximity adalah *k-Nearest Neighbors (k-NN) Outlier Detection*, 
di mana jarak ke tetangga terdekat digunakan sebagai skor anomaly. 
Semakin besar jaraknya, semakin besar kemungkinan titik tersebut merupakan anomaly.

Pendekatan ini intuitif dan tidak membutuhkan asumsi distribusi, namun performanya dapat terpengaruh oleh dimensi data yang tinggi.


### 2.4 Density Approach
Pendekatan density mendeteksi anomaly berdasarkan tingkat kepadatan lokal suatu titik. 
Jika suatu titik memiliki kepadatan yang jauh lebih rendah dibandingkan tetangga sekitarnya, maka titik tersebut dianggap sebagai outlier.

Metode yang umum digunakan adalah *Local Outlier Factor (LOF)*, 
yang menghitung perbandingan local reachability density antara sebuah titik dengan tetangganya. 
Nilai LOF lebih besar dari 1 menunjukkan bahwa titik tersebut memiliki kepadatan lebih rendah dan berpotensi menjadi anomaly.

Pendekatan ini fleksibel untuk data yang memiliki variasi kepadatan, namun lebih mahal secara komputasi dibandingkan pendekatan statistik sederhana.


Dengan memahami ketiga pendekatan tersebut, langkah selanjutnya adalah menerapkannya pada dataset Credit Card Fraud Detection serta membandingkan performanya dalam mendeteksi transaksi yang tidak normal.


## BAB 3 — Exploratory Data Analysis (EDA) dan Preprocessing

Pada bagian ini dilakukan eksplorasi awal terhadap dataset untuk memahami struktur data,
mengecek nilai kosong, serta melihat distribusi label. Tahapan EDA dilakukan secara sederhana
karena fokus utama tugas adalah implementasi tiga pendekatan anomaly detection.


In [34]:
import pandas as pd

df = pd.read_csv("data/creditcard.csv")

df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [35]:
df.isnull().sum()


Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [36]:
df['Class'].value_counts(normalize=True)


Class
0    0.998273
1    0.001727
Name: proportion, dtype: float64

### 4.1 Statistical Approach

Metode yang digunakan adalah Elliptic Envelope, yang memodelkan distribusi data normal
berdasarkan mean dan covariance. Titik yang berada di luar batas elips diprediksi sebagai
outlier (fraud).

Pada tahap implementasi:
- Data dinormalisasi menggunakan StandardScaler.
- Model dilatih menggunakan seluruh fitur kecuali label.
- Proporsi fraud pada data training digunakan sebagai parameter contamination.

Hasil prediksi diubah dari label model:
-1 → 1 (fraud)  
 1 → 0 (normal)

Metrik evaluasi yang dihitung meliputi akurasi, precision, recall, serta waktu komputasi.
Interpretasi disampaikan pada bagian analisis hasil.


In [37]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
X = df.drop(columns=["Class"])
y = df["Class"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X['Amount'] = np.log1p(X['Amount'])
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)


In [38]:
import time
import numpy as np
from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import accuracy_score, precision_score, recall_score

# contamination berdasarkan proporsi fraud di training
contamination = y_train.mean()
contamination


np.float64(0.0017254870488152324)

In [39]:
stat_model = EllipticEnvelope(
    contamination=contamination,
    random_state=42
)


In [40]:
t0 = time.time()
stat_model.fit(X_train_s)
fit_time_stat = time.time() - t0
fit_time_stat


/home/themechanic/envs/ML-project/lib/python3.12/site-packages/sklearn/covariance/_robust_covariance.py:185: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-277.170309786192490 > -278.573299410424056). You may want to try with a higher value of support_fraction (current value: 0.501).
  warnings.warn(
/home/themechanic/envs/ML-project/lib/python3.12/site-packages/sklearn/covariance/_robust_covariance.py:185: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-273.403645766972204 > -277.620159731646538). You may want to try with a higher value of support_fraction (current value: 0.501).
  warnings.warn(
/home/themechanic/envs/ML-project/lib/python3.12/site-packages/sklearn/covariance/_robust_covariance.py:185: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-185.237800087885802 > -185.928870895933301). You may want to try with a higher value of

22.15370488166809

In [41]:
t0 = time.time()
pred_stat = stat_model.predict(X_test_s)
pred_time_stat = time.time() - t0

# konversi:
# -1 (outlier) -> 1 (fraud)
#  1 (inlier)  -> 0 (normal)
pred_stat_bin = np.where(pred_stat == -1, 1, 0)


In [42]:
acc_stat = accuracy_score(y_test, pred_stat_bin)
prec_stat = precision_score(y_test, pred_stat_bin, zero_division=0)
rec_stat = recall_score(y_test, pred_stat_bin, zero_division=0)

acc_stat, prec_stat, rec_stat, fit_time_stat, pred_time_stat


(0.9970389616469459,
 0.1568627450980392,
 0.16216216216216217,
 22.15370488166809,
 0.05080914497375488)

# 4.1 Proximity Approach (kNN-based Outlier Detection)

Tahap Implementasi:
1. Menentukan K= 5
2. Menghitung jarak ke 5 tetangga
3. Merata-ratakan jarak ke tetangganya
4. memasang threshold sebesar kuartil ke (1- nilai kontaminasi)
5. Jika jarak lebih besar dari threshold maka dia fraud dan jika tidak maka dia normal

In [43]:
from sklearn.neighbors import NearestNeighbors

In [44]:
k = 5
t0 = time.time()
knn_model = NearestNeighbors(n_neighbors=k)
knn_model.fit(X_train_s)
fit_time_knn = time.time() - t0
distances_train, _ = knn_model.kneighbors(X_train_s)
avg_dist_train = distances_train.mean(axis=1)
threshold = np.quantile(avg_dist_train, 1 - contamination)
distances_test, _ = knn_model.kneighbors(X_test_s)
avg_dist_test = distances_test.mean(axis=1)
t0 = time.time()
pred_knn_bin = (avg_dist_test > threshold).astype(int)
pred_time_knn = time.time() - t0


In [45]:
acc_knn = accuracy_score(y_test, pred_knn_bin)
prec_knn = precision_score(y_test, pred_knn_bin, zero_division=0)
rec_knn = recall_score(y_test, pred_knn_bin, zero_division=0)

print(f"k-NN Accuracy  : {acc_knn:.4f}")
print(f"k-NN Precision : {prec_knn:.4f}")
print(f"k-NN Recall    : {rec_knn:.4f}")

k-NN Accuracy  : 0.9947
k-NN Precision : 0.0744
k-NN Recall    : 0.1824


# 4.2 Density Approach (Local Outlier Factor)

Tahap Implementasi:
1. membuat sebuah variabel untuk menyimpan model LocalOutlierFactor dengan tetangga 20 novelty true untuk mendeteksi data baru.


cara kerja model ini simple. mereka menghitung kepadatan lokalnya terlebih dahulu. jarak 10 akan dianggap normal jika berada di kepadatan lokal yang rendah dibandingkan kepadatan lokal yang rapat

In [46]:
from sklearn.neighbors import LocalOutlierFactor

In [47]:
t0 = time.time()
lof_model = LocalOutlierFactor(
    n_neighbors=20, 
    contamination=contamination, 
    novelty=True 
)

lof_model.fit(X_train_s)
fit_time_lof = time.time() - t0

In [48]:
t0 = time.time()
pred_lof = lof_model.predict(X_test_s)
pred_time_lof = time.time() - t0
pred_lof_bin = np.where(pred_lof == -1, 1, 0)

In [49]:
acc_lof = accuracy_score(y_test, pred_lof_bin)
prec_lof = precision_score(y_test, pred_lof_bin, zero_division=0)
rec_lof = recall_score(y_test, pred_lof_bin, zero_division=0)

print(f"LOF Accuracy  : {acc_lof:.4f}")
print(f"LOF Precision : {prec_lof:.4f}")
print(f"LOF Recall    : {rec_lof:.4f}")

LOF Accuracy  : 0.9967
LOF Precision : 0.0000
LOF Recall    : 0.0000


# Perbandingan ketiga approach

In [50]:

results = pd.DataFrame({
    'Method': ['Statistical (Elliptic)', 'Density (LOF)', 'Proximity (k-NN)'],
    'Accuracy': [acc_stat, acc_lof, acc_knn],
    'Precision': [prec_stat, prec_lof, prec_knn],
    'Recall': [rec_stat, rec_lof, rec_knn],
    'Fit Time (s)': [fit_time_stat, fit_time_lof, fit_time_knn],
    'Pred Time (s)': [pred_time_stat, pred_time_lof, pred_time_knn]
})

# Tampilkan tabel
results.sort_values(by='Recall', ascending=False)

,Method,Accuracy,Precision,Recall,Fit Time (s),Pred Time (s)
2,Proximity (k-NN),0.994651,0.074380,0.182432,0.014985,0.000353
0,Statistical (Elliptic),0.997039,0.156863,0.162162,22.153705,0.050809
1,Density (LOF),0.996664,0.000000,0.000000,41.395913,14.237450


## BAB 5 — Analisis Hasil

Berdasarkan hasil evaluasi tiga pendekatan anomaly detection pada dataset Credit Card Fraud Detection,
terlihat bahwa setiap metode menunjukkan karakteristik performa yang berbeda, terutama pada recall,
precision, dan waktu komputasi.

### 5.1 Analisis Performa

1. **Proximity Approach (k-NN)**  
   Metode k-NN menghasilkan *recall* tertinggi (0.182432).  
   Ini berarti lebih banyak transaksi fraud yang berhasil tertangkap dibandingkan dua metode lainnya.  
   Hal ini selaras dengan sifat k-NN yang sensitif terhadap titik yang jauh dari tetangganya.  
   Namun, precision-nya rendah (0.074380), menunjukkan banyak false positive.  
   Dari sisi waktu, metode ini sangat efisien dengan waktu prediksi tercepat (0.000353 detik).

2. **Statistical Approach (Elliptic Envelope)**  
   Metode ini menempati posisi tengah dengan recall sebesar 0.162162 dan precision lebih tinggi (0.156863)
   dibanding k-NN. Ini menunjukkan pendekatan statistik lebih selektif dalam menandai anomaly.  
   Fit time cukup tinggi (22.153 seconds) karena perhitungan mean dan covariance multidimensi.

3. **Density Approach (LOF)**  
   LOF memberikan performa paling rendah, dengan recall dan precision sama-sama 0.0.  
   Penyebabnya adalah LOF mengandalkan variasi kepadatan lokal, sementara dataset Credit Card Fraud 
   sudah melalui PCA sehingga distribusi fitur menjadi cukup homogen. LOF tidak menemukan perbedaan jelas
   antara titik fraud dan non-fraud. Selain itu, LOF juga memiliki waktu komputasi terlama.

### 5.2 Pola Perbedaan Metode

- **k-NN unggul recall**  
  Cocok untuk tujuan “menangkap sebanyak mungkin fraud”, meskipun banyak false alarm.

- **Elliptic Envelope lebih stabil**  
  Precision lebih tinggi karena pendekatan statistik lebih konservatif dan tidak terlalu agresif.

- **LOF tidak cocok untuk dataset PCA**  
  Density-based methods bekerja baik pada data dengan kepadatan yang bervariasi,  
  bukan data high-dimensional yang sudah ditransformasi.

Secara keseluruhan, perilaku dataset menunjukkan bahwa metode proximity dan statistik lebih sesuai 
dibanding metode density untuk kasus anomaly detection pada fraud.


## BAB 6 — Kesimpulan

Dari hasil eksperimen tiga pendekatan anomaly detection pada dataset Credit Card Fraud Detection,
dapat disimpulkan hal-hal sebagai berikut:

1. **Proximity-based k-NN merupakan metode dengan recall terbaik**, sehingga paling efektif
   jika tujuan utama adalah menangkap sebanyak mungkin transaksi fraud.

2. **Pendekatan Statistical (Elliptic Envelope) memberikan keseimbangan precision dan recall
   yang lebih baik**, cocok jika ingin mengurangi false positive.

3. **Density-based LOF tidak cocok**, terutama karena fitur PCA menyebabkan kepadatan antar titik
   menjadi relatif seragam, sehingga LOF gagal membedakan fraud dan non-fraud.

4. Pemilihan metode anomaly detection harus mempertimbangkan bentuk distribusi data,
   transformasi fitur, serta tujuan analisis (mau memaksimalkan recall atau precision).

Dengan demikian, untuk kasus fraud detection pada dataset ini:
- **k-NN adalah pilihan terbaik untuk mendeteksi sebanyak mungkin fraud (high recall)**  
- **Elliptic Envelope adalah pilihan terbaik untuk prediksi yang lebih stabil (higher precision)**


## BAB 7 — Penutup

Tugas ini memberikan pemahaman mengenai konsep dasar anomaly detection serta perbedaan
pendekatan yang digunakan untuk mendeteksi titik yang tidak mengikuti pola umum.
Melalui implementasi tiga metode—statistical, proximity, dan density—terlihat bahwa
karakteristik dataset sangat mempengaruhi performa model.

Dataset Credit Card Fraud Detection menunjukkan bahwa metode proximity dan statistical
lebih efektif dibanding density-based, terutama karena transformasi PCA yang mengurangi
variasi kepadatan lokal.

Diharapkan laporan ini dapat menjadi referensi dalam memahami anomaly detection
dan bagaimana memilih metode yang sesuai berdasarkan struktur data dan tujuan analisis.
